In [61]:
import numpy as np

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, RNN, LSTM
from torch.nn.functional import softmax, relu

from sklearn.manifold import TSNE
import h5py

In [62]:
# Load data
with h5py.File('C:/Users/jovog/OneDrive/Documents/50000_dataset.hdf5', 'r') as hf:
    data = hf['data'][:]
    label = hf['label'][:]
data[:,1] += 1
label[:] += 1

In [63]:
seq = [] #Initalize
lab = []
current_user = -1
for i in range(data.shape[0]): # going through the data and sorting into user sequences
    if data[i,0]!=current_user:
        current_user = data[i,0]
        seq.append([data[i,1]])
        lab.append([label[i]])        
    else:
        seq[-1].append(data[i,1]) # new user line
        lab[-1].append(label[i])
        
seq.sort(key=lambda x:len(x))
lab.sort(key=lambda x:len(x))
seq_length = [len(x) for x in seq]

In [64]:
# Define network
output_size = 17770
num_input = 17770
hidden_size = 100

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.one_hot = nn.Embedding(num_input, num_input)
        self.one_hot.weight.data = torch.eye(num_input)
        self.one_hot.weight.detach_()
        self.rnn = LSTM(input_size=num_input,
                        hidden_size=100,
                        num_layers=1,
                        bidirectional=False,
                        batch_first = True)
        self.l_out = Linear(in_features=hidden_size,
                            out_features=output_size,
                            bias=False)
    
    def forward(self, x):
        out = {}
        x = self.one_hot(x)
        
        x, hn = self.rnn(x)
        
        out = softmax(self.l_out(x), dim=2)
        return out
    
    def MyLoss(self, Y_hat, Y):
        
        Y = self.one_hot(Y)
        
        Y = Y.view(-1,output_size)
        
        Y_hat = Y_hat.view(-1, output_size)
        
        # create a mask by filtering out all tokens that ARE NOT the padding token
        mask = (Y > 0).float()
        print(mask.size())
        # count how many tokens we have
        #nb_tokens = int(torch.sum(mask).data[0])

        # pick the values for the label and zero out the rest with the mask
        Y_hat = Y_hat * mask
        
        return loss
net = Net()
print(net)

Net(
  (one_hot): Embedding(17770, 17770)
  (rnn): LSTM(17770, 100, batch_first=True)
  (l_out): Linear(in_features=100, out_features=17770, bias=False)
)


In [65]:
{p[0]: p[1].requires_grad for p in net.named_parameters()}

{'one_hot.weight': False,
 'rnn.weight_ih_l0': True,
 'rnn.weight_hh_l0': True,
 'rnn.bias_ih_l0': True,
 'rnn.bias_hh_l0': True,
 'l_out.weight': True}

In [66]:
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [67]:
# Train and validation loop
batch_size = 10
net.train()
for i in np.random.choice(len(seq)//batch_size,len(seq)//batch_size, replace=False):
    batch = seq[i*batch_size:(i+1)*batch_size]
    batch_length = seq_length[i*batch_size:(i+1)*batch_size]
    batch_label = lab[i*batch_size:(i+1)*batch_size]
    
    # Zero padding of the batch
    max_len_batch = max(batch_length)
    batch_padded = np.zeros((batch_size,max_len_batch))
    padded_label = np.zeros((batch_size,max_len_batch))
    for j in range(batch_size):
        batch_padded[j,0:batch_length[j]] = batch[j]
        padded_label[j,0:batch_length[j]] = batch_label[j]
    batch_padded = torch.from_numpy(batch_padded).long()
    padded_label = torch.from_numpy(padded_label).long()
    output = net(batch_padded)
    batch_loss = net.MyLoss(output,padded_label)
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

tensor([ 6134,   550,  1073,  1144,  1145,  1307,  1659,  1798,  2152,  3427,
         3610,  3736,  4520,  9144,  5293,  5317,  5496,  6073,  6263,  6287,
         6692,  6972,  7234,  9340,  9635,  9658, 10255, 11149, 12299, 12317,
        12435, 12470, 12672, 12911, 13651, 14149, 14233, 14313, 14367, 15124,
        15205, 15582, 16128, 16242, 16377, 16384, 16954, 17057, 17324,  5862,
        10906,  5730, 16357, 16469,   406,  1719,  2495,  3138,  3418,  3433,
         3825,  3936,  4389,  4745,  4883,  5239,  5327,  5939,  6497,  7240,
         7280,  7624,  8636,  9617, 10170, 10231, 10397, 10625, 11040, 11812,
        11837, 12092, 12232, 12494, 13255, 13462, 13524, 14660, 14718, 15151,
        15393, 15887, 16339, 17149, 17293])
tensor([  550,  1073,  1144,  1145,  1307,  1659,  1798,  2152,  3427,  3610,
         3736,  4520,  9144,  5293,  5317,  5496,  6073,  6263,  6287,  6692,
         6972,  7234,  9340,  9635,  9658, 10255, 11149, 12299, 12317, 12435,
        12470, 12672

In [26]:
print(output)

tensor([[[0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         ...,
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001]],

        [[0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         ...,
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001]],

        [[0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
         [0.0001, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.